# Team Assignment - Cohort A
First we pick a seed for a complete random assignment:

In [0]:
seed = 42
n_leads_needed = 0  # beyond the volunteer leads

In [0]:
!pip install --upgrade -q gspread

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
worksheet = gc.open('Teaming-IS833-Spring20').sheet1

In [0]:
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
students = pd.DataFrame.from_records(rows)
students.columns = ['name', 'email', 'team']
students['team'] = students['team'].astype('bool')
students.head(5)

## Leads
Volunteer leads:

In [0]:
volunteer_leads = students[students['team'] == True][['name']]
volunteer_leads

Leads picked by a random generator:

In [0]:
random_leads = students[students['team'] == False][['name']].sample(n=n_leads_needed, random_state=seed)['name']
leads = pd.concat([volunteer_leads, random_leads])
leads = leads.sample(frac=1, random_state=seed).reset_index()[['name']]

leads = pd.DataFrame({'name': leads['name'].tolist()}, index=pd.Series(range(1,9)))
leads.index.name = 'team'
print('List of leads:')
leads

In [0]:
leads_l = list(leads['name'])
students.loc[students['name'].isin(leads_l), 'team'] = True
teams = pd.DataFrame()

for i, leader in enumerate(leads_l):
  take_n = 4
  if i < (students.shape[0] % (take_n+1)): take_n += 1
  lead_ = students[students['name'] == leader][['name', 'email']]
  team_tmp = students[students['team'] == False][['name', 'email']].sample(n=take_n, random_state=seed)
  students.loc[students['name'].isin(team_tmp['name']), 'team'] = True
  team_tmp = pd.concat([lead_, team_tmp])
  team_tmp['team'] = i+1
  
  teams = pd.concat([teams, team_tmp])
teams = teams.reset_index(drop=True)
print(teams)

In [0]:
teams.to_csv('teams.csv', index=False)